In [46]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import pytz
import duckdb
import pyCLIF as pc
import warnings
warnings.filterwarnings("ignore")

## Base Population

#### ADT

In [2]:
adt = pc.load_data("clif_adt")
adt["hospitalization_id"] = adt["hospitalization_id"].astype(str)
adt = pc.convert_datetime_columns_to_site_tz(adt, pc.helper["your_site_timezone"])
adt["in_dttm"] = pc.getdttm(adt["in_dttm"])
pc.deftime(adt["in_dttm"])
adt.head()

Data loaded successfully from C:/Users/vchaudha/Downloads/rush_parquet/clif_adt.parquet
in_dttm: null count = 0
out_dttm: null count = 0
Count with hours and minutes: 1135203
Count without hours and minutes: 0


,hospitalization_id,event_type,location_name,in_dttm,out_dttm,location_category,hospital_id
0,100113836932,Admission,Rush Univeristy Antepartum Postpartum,2018-05-03 22:34:00,2018-05-04 18:37:00-05:00,ward,RUMC
1,100113836932,Transfer In,Labor & Delivery,2018-05-04 18:37:00,2018-05-05 01:59:00-05:00,l&d,RUMC
2,100113836932,Transfer In,Rush Univeristy Antepartum Postpartum,2018-05-05 01:59:00,2018-05-06 12:20:00-05:00,ward,RUMC
3,100113836932,Discharge,Rush Univeristy Antepartum Postpartum,2018-05-06 12:20:00,1900-01-01 00:00:00-05:51,ward,RUMC
4,100125123564,Transfer In,Labor & Delivery,2018-04-26 23:01:00,2018-04-26 23:25:00-05:00,l&d,RUMC


#### Hospitalization

In [3]:
hosp = pc.load_data("clif_hospitalization")
hosp["hospitalization_id"] = hosp["hospitalization_id"].astype(str)
if "hospitalization_joined_id" not in hosp.columns:
    hosp["hospitalization_joined_id"] = hosp["hospitalization_id"]

hosp["hospitalization_joined_id"] = hosp["hospitalization_joined_id"].astype(str)

hosp = pc.convert_datetime_columns_to_site_tz(hosp, pc.helper["your_site_timezone"])
hosp["admission_dttm"] = pc.getdttm(hosp["admission_dttm"])
hosp["discharge_dttm"] = pc.getdttm(hosp["discharge_dttm"])

adt["Hosp_key_bkp"] = adt["hospitalization_id"]
hosp["Hosp_key_bkp"] = hosp["hospitalization_id"]

hosp.head()

Data loaded successfully from C:/Users/vchaudha/Downloads/rush_parquet/clif_hospitalization.parquet
admission_dttm: null count = 0
discharge_dttm: null count = 446


,patient_id,hospitalization_id,zipcode_five_digit,admission_dttm,discharge_dttm,discharge_name,age_at_admission,discharge_category,hospitalization_joined_id,Hosp_key_bkp
0,5282613,100207673243,60406,2022-12-27 09:00:00,2022-12-29 12:36:00,Discharged to home or self care (routine disch...,48,Home,100207673243,100207673243
1,5100571,100177118306,60620,2020-11-16 17:13:00,2020-11-19 14:22:00,Discharged to home or self care (routine disch...,72,Home,100177118306,100177118306
2,7355974,100150644186,46368,2018-03-16 21:49:00,2018-03-21 18:23:00,Discharged/transd to home (care by home health...,65,Home,100150644186,100150644186
3,7623068,100150682826,60622,2018-03-19 19:45:00,2018-03-23 14:21:00,Discharged to home or self care (routine disch...,25,Home,100150682826,100150682826
4,8203571,100216198515,60586,2023-07-19 20:56:00,2023-07-22 13:28:00,Discharged to home or self care (routine disch...,38,Home,100216198515,100216198515


#### Hospitalization Stitching

In [4]:
eblock = pc.stitch_encounters(hosp, adt)

# Create mapping dictionary
hospitalization_to_block = {
    hospital_id: block
    for block, hospital_list in zip(
        eblock["encounter_block"].astype(str), eblock["list_hospitalization_id"]
    )
    for hospital_id in hospital_list
}

In [5]:
# Convert to DataFrame
hospitalization_to_block_df = pd.DataFrame(
    list(hospitalization_to_block.items()),
    columns=["hospitalization_id", "encounter_block"],
)
hospitalization_to_block_df.to_csv(
    "../output/intermediate/hospitalization_to_block_df.csv", index=False
)

In [6]:
agg_rules_hosp = {
    "patient_id": "first",  # Assuming patient_id is consistent across duplicates
    "zipcode_five_digit": "first",  # Retain first occurrence
    "admission_dttm": "min",  # Earliest admission date
    "discharge_dttm": "max",  # Latest discharge date
    "discharge_name": "last",  # Prioritize first value (change as per logic)
    "age_at_admission": "mean",  # Take average if different
    "discharge_category": "last",  # Keep the first occurrence
    "hospitalization_joined_id": lambda x: ", ".join(
        x.unique()
    ),  # Retain first occurrence
    "Hosp_key_bkp": lambda x: ", ".join(
        x.unique()
    ),  # Backup key, take first occurrence
}

hosp["hospitalization_id"] = hosp["hospitalization_id"].map(hospitalization_to_block)
hosp["hospitalization_id"] = hosp["hospitalization_id"].astype(str)
hosp = hosp.sort_values(by=["hospitalization_id", "admission_dttm"])

hosp = hosp.groupby("hospitalization_id").agg(agg_rules_hosp).reset_index()

In [7]:
adt = adt[["hospitalization_id", "in_dttm", "location_category", "hospital_id"]]
adt["hospitalization_id"] = (
    adt["hospitalization_id"].map(hospitalization_to_block).astype(str)
)
adt = adt.sort_values(by=["hospitalization_id", "in_dttm"])

#### Respiratory Support

In [8]:
rst = pc.load_data("clif_respiratory_support")
rst["hospitalization_id"] = rst["hospitalization_id"].astype(str)
rst["hospitalization_id"] = (
    rst["hospitalization_id"]
    .map(hospitalization_to_block)
    .fillna(-1)
    .astype(int)
    .astype(str)
)
rst = rst[
    ~rst["hospitalization_id"].isin(
        rst[rst["tracheostomy"] == 1].hospitalization_id.unique()
    )
]  # exclude trach pats

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Data loaded successfully from C:/Users/vchaudha/Downloads/rush_parquet/clif_respiratory_support.parquet


In [9]:
rst = pc.convert_datetime_columns_to_site_tz(rst, pc.helper["your_site_timezone"])

recorded_dttm: null count = 0


#### Patient

In [10]:
pat = pc.load_data("clif_patient")
pat = pc.convert_datetime_columns_to_site_tz(pat, pc.helper["your_site_timezone"])

Data loaded successfully from C:/Users/vchaudha/Downloads/rush_parquet/clif_patient.parquet
death_dttm: null count = 109965


#### Cohort Filtering

In [11]:
imv_hosp_ids = rst[
    rst["device_category"].str.lower() == "imv"
].hospitalization_id.unique()
icu_hosp_ids = adt[
    adt["location_category"].str.lower() == "icu"
].hospitalization_id.unique()

icu_hosp_ids = [x for x in icu_hosp_ids if x is not None]
imv_hosp_ids = [x for x in imv_hosp_ids if x is not None]

hosp = hosp[
    (hosp["admission_dttm"].dt.year >= 2022)
    & (hosp["admission_dttm"].dt.year <= 2024)
    & (hosp["discharge_dttm"].dt.year <= 2024)
    & (hosp["hospitalization_id"].isin(np.intersect1d(imv_hosp_ids, icu_hosp_ids)))
    & (hosp["age_at_admission"] >= 18)
    & (hosp["age_at_admission"] <= 119)
].reset_index(drop=True)

required_id = hosp["hospitalization_id"].unique()
print(len(required_id), " : potential cohort count")

base = pd.merge(hosp, pat, on="patient_id", how="inner")[
    [
        "patient_id",
        "hospitalization_id",
        "admission_dttm",
        "discharge_dttm",
        "age_at_admission",
        "discharge_category",
        "sex_category",
        "race_category",
        "ethnicity_category",
        "language_name",
    ]
]

base["admission_dttm"] = pc.getdttm(base["admission_dttm"])

base.columns

adt = adt[adt["hospitalization_id"].isin(required_id)].reset_index(drop=True)
rst = rst[rst["hospitalization_id"].isin(required_id)].reset_index(drop=True)

4176  : potential cohort count


In [12]:
base.head()

,patient_id,hospitalization_id,admission_dttm,discharge_dttm,age_at_admission,discharge_category,sex_category,race_category,ethnicity_category,language_name
0,322999,10015,2022-04-03 13:21:00,2022-05-18 13:16:00,42.0,Acute Inpatient Rehab Facility,Male,Black or African American,Non-Hispanic,English
1,6640001,100201,2022-12-06 07:57:00,2022-12-07 10:05:00,40.0,Against Medical Advice (AMA),Male,White,Non-Hispanic,English
2,6640001,100202,2022-12-17 14:22:00,2023-01-05 14:53:00,40.0,Home,Male,White,Non-Hispanic,English
3,6640564,100260,2024-12-17 05:26:00,2024-12-22 11:46:00,50.0,Home,Male,White,Non-Hispanic,English
4,6641404,100298,2024-06-29 17:26:00,2024-06-30 02:25:00,67.0,Expired,Male,White,Hispanic,English


In [13]:
rst.head()

,hospitalization_id,recorded_dttm,P/F,device_name,fio2_set,flow_rate_set,inspiratory_time_set,interface,invasive_vent_off,invasive_vent_on,...,resp_rate_set,sbt_timepoint,tidal_volume_obs,tidal_volume_set,tube_comp_%,vent_brand_name,device_category,mode_category,tracheostomy,pressure_control_set
0,15858,2022-05-18 07:28:00-05:00,None,nan,NaN,NaN,NaN,None,None,None,...,NaN,None,NaN,NaN,NaN,None,None,None,0,NaN
1,15858,2022-05-18 17:19:00-05:00,None,Ventilator,NaN,NaN,NaN,None,None,None,...,NaN,None,NaN,NaN,NaN,None,IMV,None,0,NaN
2,15858,2022-05-18 17:23:00-05:00,None,nan,1.0,NaN,0.9,Invasive,None,Vent On,...,16.0,None,345.0,385.0,NaN,PB 980,None,Pressure-Regulated Volume Control,0,NaN
3,15858,2022-05-18 17:24:00-05:00,None,nan,0.8,NaN,NaN,None,None,None,...,NaN,None,NaN,NaN,NaN,None,None,None,0,NaN
4,15858,2022-05-18 17:25:00-05:00,None,nan,0.6,NaN,NaN,None,None,None,...,NaN,None,NaN,NaN,NaN,None,None,None,0,NaN


#### Water Fall Start

In [14]:
### aswaterfallneedinutc
rst['recorded_dttm'] = rst['recorded_dttm'].dt.tz_convert('UTC')

new_rst = pc.process_resp_support_waterfall(rst)
new_rst.head()

✦ Phase 0 – initialise & create hourly scaffold
✦ Phase 1 – heuristic inference of device / mode
✦ Phase 2 – build device / mode hierarchical IDs
✦ Phase 3 – numeric down/up-fill inside mode_name_id blocks
✦ Phase 4 – final deduplication & ordering
✅ Respiratory-support waterfall complete.


,hospitalization_id,recorded_dttm,P/F,device_name,fio2_set,flow_rate_set,inspiratory_time_set,interface,invasive_vent_off,invasive_vent_on,...,device_category,mode_category,tracheostomy,pressure_control_set,recorded_date,recorded_hour,device_cat_id,device_id,mode_cat_id,mode_name_id
0,10015,2022-04-04 12:45:00+00:00,None,nan,NaN,NaN,NaN,None,None,None,...,NaN,None,0.0,NaN,2022-04-04,12,1,1,1,1
1,10015,2022-04-04 14:00:00+00:00,None,nan,NaN,NaN,NaN,None,None,None,...,NaN,None,0.0,NaN,2022-04-04,14,1,1,1,1
2,10015,2022-04-04 16:15:00+00:00,None,nan,NaN,NaN,NaN,None,None,None,...,NaN,None,0.0,NaN,2022-04-04,16,1,1,1,1
3,10015,2022-04-04 21:00:00+00:00,None,nan,NaN,NaN,NaN,None,None,None,...,NaN,None,0.0,NaN,2022-04-04,21,1,1,1,1
4,10015,2022-04-05 00:45:00+00:00,None,nan,NaN,NaN,NaN,None,None,None,...,NaN,None,0.0,NaN,2022-04-05,0,1,1,1,1


In [15]:
new_rst['recorded_dttm'] = new_rst['recorded_dttm'].dt.tz_convert(pc.helper['your_site_timezone'])
rst = new_rst.copy()

rst["recorded_dttm"] = pc.getdttm(rst["recorded_dttm"])
pc.deftime(rst["recorded_dttm"])

Count with hours and minutes: 1435191
Count without hours and minutes: 0


water fall end

In [16]:
if pc.helper["site_name"] == "RUSH":
    rst_col = [
        "hospitalization_id",
        "recorded_dttm",
        "device_category",
        "mode_category",
        "fio2_set",
        "peep_set",
        "resp_rate_set",
        "pressure_support_set",
        "mode_name",
        "tube_comp_%",
        "sbt_timepoint",
        "vent_brand_name",
    ]
    rst["device_category"] = rst["device_category"].replace("nan", np.nan)
else:
    rst_col = [
        "hospitalization_id",
        "recorded_dttm",
        "device_category",
        "mode_category",
        "fio2_set",
        "peep_set",
        "resp_rate_set",
        "pressure_support_set",
        "mode_name",
        "vent_brand_name",
    ]
rst = rst[rst_col]
rst["device_category"] = rst["device_category"].str.lower()
rst["mode_category"] = rst["mode_category"].str.lower()

#### Extubated Flag

In [17]:
# Step 1: Sort and forward-fill device_category by hospitalization_id
rst = rst.sort_values(["hospitalization_id", "recorded_dttm"])
rst["device_category"] = rst.groupby("hospitalization_id")["device_category"].ffill()

# Step 2: Create a device_segment_id for each change in device_category within a hospitalization
rst["device_change"] = (
    rst["device_category"]
    != rst.groupby("hospitalization_id")["device_category"].shift()
).astype(int)
rst["device_segment_id"] = rst.groupby("hospitalization_id")["device_change"].cumsum()

# Step 3: Forward-fill mode_category within each device_segment_id and hospitalization_id
rst["mode_category"] = rst.groupby(["hospitalization_id", "device_segment_id"])[
    "mode_category"
].ffill()

# Step 4: Create Device_IMV column
rst["Device_IMV"] = (rst["device_category"] == "imv").astype(int)

# Step 5: Flag extubation when there's a switch from IMV to two consecutive non-IMV entries
rst["extubated"] = 0
for hosp_id, group in tqdm(rst.groupby("hospitalization_id")):
    group = group.sort_values("recorded_dttm")
    idx = group.index
    for i in range(len(group) - 2):
        if (
            group.iloc[i]["Device_IMV"] == 1
            and group.iloc[i + 1]["Device_IMV"] == 0
            and group.iloc[i + 2]["Device_IMV"] == 0
        ):
            rst.at[idx[i], "extubated"] = 1

100%|██████████| 4176/4176 [03:03<00:00, 22.77it/s]


In [18]:
rst.head()

,hospitalization_id,recorded_dttm,device_category,mode_category,fio2_set,peep_set,resp_rate_set,pressure_support_set,mode_name,tube_comp_%,sbt_timepoint,vent_brand_name,device_change,device_segment_id,Device_IMV,extubated
0,10015,2022-04-04 07:45:00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,None,None,1,1,0,0
1,10015,2022-04-04 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,None,None,1,2,0,0
2,10015,2022-04-04 11:15:00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,None,None,1,3,0,0
3,10015,2022-04-04 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,None,None,1,4,0,0
4,10015,2022-04-04 19:45:00,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,None,None,1,5,0,0


### MAC

In [19]:
mac = pc.load_data("clif_medication_admin_continuous")
mac["hospitalization_id"] = mac["hospitalization_id"].astype(str)
mac["hospitalization_id"] = (
    mac["hospitalization_id"].map(hospitalization_to_block).astype(str)
)
mac_col = [
    "hospitalization_id",
    "admin_dttm",
    "med_dose",
    "med_category",
    "med_dose_unit",
]
mac = mac[
    (mac["hospitalization_id"].isin(required_id))
    & (
        mac["med_category"].isin(
            [
                "norepinephrine",
                "epinephrine",
                "phenylephrine",
                "angiotensin",
                "vasopressin",
                "dopamine",
                "dobutamine",
                "milrinone",
                "isoproterenol",
                "cisatracurium",
                "vecuronium",
                "rocuronium",
                "fentanyl",
                "propofol",
                "lorazepam",
                "midazolam",
                "hydromorphone",
                "morphine",
            ]
        )
    )
][mac_col].reset_index(drop=True)


mac = pc.convert_datetime_columns_to_site_tz(mac, pc.helper["your_site_timezone"])
mac["admin_dttm"] = pc.getdttm(mac["admin_dttm"])

mac["med_dose_unit"] = mac["med_dose_unit"].str.lower()
mac = mac[
    (mac["med_dose_unit"].str.contains(r"/", na=False))
    & (mac["med_dose_unit"] != "units/hr")
].reset_index(drop=True)

Data loaded successfully from C:/Users/vchaudha/Downloads/rush_parquet/clif_medication_admin_continuous.parquet
admin_dttm: null count = 0


In [20]:
pc.deftime(mac['admin_dttm'])

Count with hours and minutes: 488363
Count without hours and minutes: 0


In [21]:
mac.head()

,hospitalization_id,admin_dttm,med_dose,med_category,med_dose_unit
0,78960,2022-10-29 04:28:00,25.0,phenylephrine,mcg/min
1,209671,2023-07-31 09:30:00,125.0,phenylephrine,mcg/min
2,191359,2022-04-19 13:30:00,75.0,phenylephrine,mcg/min
3,200973,2023-06-18 09:05:00,287.5,phenylephrine,mcg/min
4,218710,2024-11-15 16:35:00,37.5,phenylephrine,mcg/min


### Patient Assessment

In [22]:
cat_values_mapping_dict = {
    "negative": 0,
    "fail": 0,
    "pass": 1,
    "positive": 1,
    None: np.nan,
    np.nan: np.nan,
    "yes": 1,
    "no": 0,
}

pat_assess_cats_rquired = [
    "sbt_delivery_pass_fail",
    "sbt_screen_pass_fail",
    "sat_delivery_pass_fail",
    "sat_screen_pass_fail",
    "gcs_total",
    "rass",
]

In [23]:
pat_at = pc.load_data("clif_patient_assessments", -1)
pat_at_col = [
    "hospitalization_id",
    "recorded_dttm",
    "numerical_value",
    "categorical_value",
    "assessment_category",
]
pat_at["assessment_category"] = pat_at["assessment_category"].str.lower()
pat_at = pat_at[(pat_at["assessment_category"].isin(pat_assess_cats_rquired))][
    pat_at_col
].reset_index(drop=True)

pat_at = pc.convert_datetime_columns_to_site_tz(pat_at, pc.helper["your_site_timezone"])
pat_at["recorded_dttm"] = pc.getdttm(pat_at["recorded_dttm"])
pc.deftime(pat_at["recorded_dttm"])

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Data loaded successfully from C:/Users/vchaudha/Downloads/rush_parquet/clif_patient_assessments.parquet
recorded_dttm: null count = 0
Count with hours and minutes: 4386377
Count without hours and minutes: 0


In [24]:
pat_at["hospitalization_id"] = pat_at["hospitalization_id"].astype(str)
pat_at["hospitalization_id"] = (
    pat_at["hospitalization_id"]
    .map(hospitalization_to_block)
    .fillna(-1)
    .astype(int)
    .astype(str)
)

pat_at = pat_at[(pat_at["hospitalization_id"].isin(required_id))][
    pat_at_col
].reset_index(drop=True)
pat_at["categorical_value"] = (
    pat_at["categorical_value"].str.lower().map(cat_values_mapping_dict)
)
pat_at["assessment_value"] = pat_at["numerical_value"].combine_first(
    pat_at["categorical_value"]
)
pat_at.drop(columns=["numerical_value", "categorical_value"], inplace=True)

In [25]:
pat_at.assessment_category.unique()

array(['sbt_delivery_pass_fail', 'gcs_total', 'sat_delivery_pass_fail',
       'sat_screen_pass_fail', 'sbt_screen_pass_fail', 'rass'],
      dtype=object)

In [26]:
pat_at['assessment_value'].value_counts()

assessment_value
0      169487
15      78659
-1      52798
+1      35935
-2      29335
-4      23873
+2      22965
-3      22537
14      22063
1.0     19972
10      16139
-5      14997
11      13690
3       11071
8       10074
9        9619
7        9183
0.0      8751
13       7898
6        7086
5        6068
+3       4758
12       4628
4        4450
+4        720
Name: count, dtype: int64

In [27]:
pat_at.head()

,hospitalization_id,recorded_dttm,assessment_category,assessment_value
0,159939,2024-02-24 12:25:00,sbt_delivery_pass_fail,0.0
1,12786,2024-10-29 04:40:00,sbt_delivery_pass_fail,0.0
2,36280,2024-10-09 08:53:00,sbt_delivery_pass_fail,0.0
3,59468,2024-01-03 01:39:00,sbt_delivery_pass_fail,0.0
4,156310,2023-11-15 04:36:00,sbt_delivery_pass_fail,0.0


### Vitals

In [28]:
vit = pc.load_data("clif_vitals", -1)
vit["hospitalization_id"] = vit["hospitalization_id"].astype(str)
vit["hospitalization_id"] = (
    vit["hospitalization_id"].map(hospitalization_to_block).astype(str)
)
vit_col = ["hospitalization_id", "recorded_dttm_min", "vital_category", "vital_value"]
vit["vital_category"] = vit["vital_category"].str.lower()


vit = pc.convert_datetime_columns_to_site_tz(vit, pc.helper["your_site_timezone"])

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Data loaded successfully from C:/Users/vchaudha/Downloads/rush_parquet/clif_vitals.parquet
recorded_dttm: null count = 7


In [29]:
vit["recorded_dttm_min"] = pc.getdttm(vit["recorded_dttm"])
pc.deftime(vit["recorded_dttm_min"])
vit = vit[
    (vit["hospitalization_id"].isin(required_id))
    & (
        vit["vital_category"].isin(
            [
                "map",
                "heart_rate",
                "sbp",
                "dbp",
                "spo2",
                "respiratory_rate",
                "weight_kg",
                "height_cm",
            ]
        )
    )
][vit_col].reset_index(drop=True)

# Sort by hospitalization_id and recorded_dttm
vit = vit.sort_values(by=["hospitalization_id", "recorded_dttm_min"])

# Group by hospitalization_id, vital_category, and recorded_dttm_min, then take the first occurrence of vital_value
vit = vit.groupby(
    ["hospitalization_id", "vital_category", "recorded_dttm_min"], as_index=False
).agg({"vital_value": "first"})
# make sure float
vit["vital_value"] = vit["vital_value"].astype(float)

# for meds
vit_weight = vit[vit["vital_category"] == "weight_kg"].reset_index(drop=True)

Count with hours and minutes: 85346323
Count without hours and minutes: 7


In [30]:
vit.head()

,hospitalization_id,vital_category,recorded_dttm_min,vital_value
0,10015,dbp,2022-04-03 15:15:00,63.0
1,10015,dbp,2022-04-03 19:45:00,67.0
2,10015,dbp,2022-04-03 23:15:00,58.0
3,10015,dbp,2022-04-04 03:15:00,55.0
4,10015,dbp,2022-04-04 07:45:00,58.0


In [31]:
# Count duplicates
duplicates = vit.duplicated(
    subset=["hospitalization_id", "vital_category", "recorded_dttm_min"], keep=False
)

# Show any duplicates (should be empty if grouping worked correctly)
vit[duplicates]

,hospitalization_id,vital_category,recorded_dttm_min,vital_value


### Med Unit Conversion

In [32]:
vit_weight.rename(
    {"vital_category": "med_category", "recorded_dttm_min": "admin_dttm"},
    axis="columns",
    inplace=True,
)

new_mac = pd.concat([mac, vit_weight], ignore_index=True)

new_mac = new_mac.sort_values(by=["hospitalization_id", "admin_dttm"])

new_mac["vital_value"] = (
    new_mac.groupby("hospitalization_id")["vital_value"].ffill().bfill()
)

new_mac = new_mac[~(new_mac["med_category"] == "weight_kg")].reset_index(drop=True)

print("mac rows:", mac.shape, "New mac rows:", new_mac.shape)
# del vit_weight

mac rows: (488363, 5) New mac rows: (488363, 6)


In [33]:
new_mac.head(5)

,hospitalization_id,admin_dttm,med_dose,med_category,med_dose_unit,vital_value
0,10015,2022-04-06 10:00:00,10.00,dobutamine,mcg/kg/min,78.5622
1,10015,2022-04-06 11:00:00,0.00,dobutamine,mcg/kg/min,78.5622
2,10015,2022-04-17 20:30:00,0.03,vasopressin,units/min,78.1086
3,10015,2022-04-18 03:08:00,30.00,propofol,mcg/kg/min,78.1086
4,10015,2022-04-18 03:08:00,30.00,propofol,mcg/kg/min,78.1086


In [34]:
# The med_unit_info dictionary
med_unit_info = {
    'norepinephrine': {
        'required_unit': 'mcg/kg/min',
        'acceptable_units': ['mcg/kg/min', 'mcg/kg/hr', 'mg/kg/hr', 'mcg/min', 'mg/hr'],
    },
    'epinephrine': {
        'required_unit': 'mcg/kg/min',
        'acceptable_units': ['mcg/kg/min', 'mcg/kg/hr', 'mg/kg/hr', 'mcg/min', 'mg/hr'],
    },
    'phenylephrine': {
        'required_unit': 'mcg/kg/min',
        'acceptable_units': ['mcg/kg/min', 'mcg/kg/hr', 'mg/kg/hr', 'mcg/min', 'mg/hr'],
    },
    'angiotensin': {
        'required_unit': 'ng/kg/min',
        'acceptable_units': ['ng/kg/min', 'ng/kg/hr'],
    },
    'vasopressin': {
        'required_unit': 'units/min',
        'acceptable_units': ['units/min', 'units/hr', 'milliunits/min', 'milliunits/hr'],
    },
    'dopamine': {
        'required_unit': 'mcg/kg/min',
        'acceptable_units': ['mcg/kg/min', 'mcg/kg/hr', 'mg/kg/hr', 'mcg/min', 'mg/hr'],
    },
    'dobutamine': {
        'required_unit': 'mcg/kg/min',
        'acceptable_units': ['mcg/kg/min', 'mcg/kg/hr', 'mg/kg/hr', 'mcg/min', 'mg/hr'],
    },
    'milrinone': {
        'required_unit': 'mcg/kg/min',
        'acceptable_units': ['mcg/kg/min', 'mcg/kg/hr', 'mg/kg/hr', 'mcg/min', 'mg/hr'],
    },
    'isoproterenol': {
        'required_unit': 'mcg/kg/min',
        'acceptable_units': ['mcg/kg/min', 'mcg/kg/hr', 'mg/kg/hr', 'mcg/min', 'mg/hr'],
    },
}

def convert_med_dose(row):
    category = row['med_category']
    # If the category is not in our dictionary, skip conversion.
    if category not in med_unit_info:
        return row
    
    info = med_unit_info[category]
    required_unit = info['required_unit']
    acceptable_units = info['acceptable_units']
    
    current_unit = row['med_dose_unit']
    dose = row['med_dose']
    weight = row['vital_value']  # patient's weight in kg

    # If the current unit already matches the required unit, nothing to do.
    if current_unit == required_unit:
        return row

    # If the current unit is not in the acceptable list, skip conversion.
    if current_unit not in acceptable_units:
        return row

    # Start with a conversion factor of 1.
    conversion_factor = 1.0

    # --------------------------------------------------
    # 1. Weight conversion: if the current unit is per kg but the required is not,
    # then multiply by the patient’s weight.
    if 'kg' in current_unit and 'kg' not in required_unit:
        conversion_factor *= weight
    elif 'kg' not in current_unit and 'kg' in required_unit:
        conversion_factor /= weight

    # --------------------------------------------------
    # 2. Time conversion: convert from per hour to per minute or vice versa.
    if 'hr' in current_unit and 'min' in required_unit:
        conversion_factor /= 60.0
    elif 'min' in current_unit and 'hr' in required_unit:
        conversion_factor *= 60.0

    # --------------------------------------------------
    # 3. Medication unit conversion (e.g., mg to mcg, milliunits to units)
    # We assume the first part (before the first '/') is the measurement unit.
    current_med_unit = current_unit.split('/')[0]
    required_med_unit = required_unit.split('/')[0]

    med_conversion = {
        ('mg', 'mcg'): 1000,
        ('mcg', 'mg'): 0.001,
        ('milliunits', 'units'): 0.001,
        ('units', 'milliunits'): 1000,
    }

    if current_med_unit != required_med_unit:
        factor = med_conversion.get((current_med_unit, required_med_unit))
        if factor is not None:
            conversion_factor *= factor
        else:
            # If no conversion factor is defined, skip conversion.
            return row

    # --------------------------------------------------
    # Apply the conversion
    new_dose = dose * conversion_factor

    # Update the row with the converted dose and unit.
    row['med_dose'] = new_dose
    row['med_dose_unit'] = required_unit
    return row

# Apply the conversion function with tqdm for progress tracking
tqdm.pandas(desc="Converting medication doses")
new_mac = new_mac.progress_apply(convert_med_dose, axis=1)

Converting medication doses: 100%|██████████| 488363/488363 [00:43<00:00, 11116.29it/s]


In [35]:
# Create a summary table for each med_category
summary_table = (
    new_mac.groupby(["med_category", "med_dose_unit"])
    .agg(
        total_N=("med_category", "size"),
        min=("med_dose", "min"),
        max=("med_dose", "max"),
        first_quantile=("med_dose", lambda x: x.quantile(0.25)),
        second_quantile=("med_dose", lambda x: x.quantile(0.5)),
        third_quantile=("med_dose", lambda x: x.quantile(0.75)),
        missing_values=("med_dose", lambda x: x.isna().sum()),
    )
    .reset_index()
)

## check the distrbituon of required continuous meds
summary_table

,med_category,med_dose_unit,total_N,min,max,first_quantile,second_quantile,third_quantile,missing_values
0,cisatracurium,mcg/kg/min,988,0.0,9.000000,3.000000,3.000000,3.000000,0
1,dobutamine,mcg/kg/min,8281,0.0,30.000000,2.000000,2.500000,4.000000,0
2,dopamine,mcg/kg/min,360,0.0,40.000000,2.500000,5.000000,7.500000,0
3,epinephrine,mcg/kg/min,13408,0.0,2.770620,0.039255,0.132452,0.300630,0
4,fentanyl,mcg/hr,54642,0.0,13750.000000,25.000000,100.000000,175.000000,0
5,fentanyl,mcg/kg/hr,1,5.0,5.000000,5.000000,5.000000,5.000000,0
6,hydromorphone,mg/hr,2495,0.0,40.000000,1.000000,3.000000,4.500000,0
7,isoproterenol,mcg/kg/min,3,0.0,0.045802,0.015267,0.030535,0.038168,0
8,lorazepam,mg/hr,12,0.0,4.000000,0.000000,2.000000,4.000000,0
9,midazolam,mg/hr,2218,0.0,50.000000,2.000000,4.500000,9.000000,0


## Wide Dataset

In [36]:
duckdb.register("base", base)
duckdb.register("pat_at", pat_at)
duckdb.register("rst", rst)
duckdb.register("mac", new_mac)
duckdb.register('adt',adt)
duckdb.register('vit',vit)

q="""
WITH
    uni_event_dttm as (
        select distinct
            hospitalization_id,
            event_time
        from
            (
                SELECT
                    hospitalization_id,
                    in_dttm AS event_time
                FROM
                    adt
                where
                    in_dttm is not null
                UNION
                SELECT
                    hospitalization_id,
                    recorded_dttm AS event_time
                FROM
                    rst
                where
                    recorded_dttm is not null
                UNION
                SELECT
                    hospitalization_id,
                    recorded_dttm AS event_time
                FROM
                    pat_at
                where
                    recorded_dttm is not null
                UNION
                SELECT
                    hospitalization_id,
                    admin_dttm AS event_time
                FROM
                    mac
                where
                    admin_dttm is not null
                UNION
                SELECT
                    hospitalization_id,
                    recorded_dttm_min AS event_time
                FROM
                    vit
                where
                    recorded_dttm_min is not null
            ) uni_time
    )
select distinct
    patient_id,
    a.hospitalization_id,
    admission_dttm,
    discharge_dttm,
    age_at_admission,
    discharge_category,
    sex_category,
    race_category,
    ethnicity_category,
    language_name,
    event_time
from
    base a
    left join uni_event_dttm b on a.hospitalization_id = b.hospitalization_id
"""
wide_cohort_df = duckdb.sql(q).df()
pc.deftime(wide_cohort_df['event_time'])

Count with hours and minutes: 1824960
Count without hours and minutes: 0


#### pivots for assessment and mac table 

In [37]:
query = """
WITH pas_data AS (
    SELECT  distinct assessment_value ,	assessment_category	,
    hospitalization_id || '_' || strftime(recorded_dttm, '%Y%m%d%H%M') AS combo_id
    FROM pat_at where recorded_dttm is not null 
) 
PIVOT pas_data
ON assessment_category
USING first(assessment_value)
GROUP BY combo_id
"""
p_pas = duckdb.sql(query).df()

query = """
WITH mac_data AS (
    SELECT  distinct med_dose ,	med_category	,
    hospitalization_id || '_' || strftime(admin_dttm, '%Y%m%d%H%M') AS combo_id
    FROM mac where admin_dttm is not null 
) 
PIVOT mac_data
ON med_category
USING min(med_dose)
GROUP BY combo_id
"""
p_mac = duckdb.sql(query).df()



In [38]:

query = """
WITH vital_data AS (
    SELECT  distinct vital_category,	vital_value	,
    hospitalization_id || '_' || strftime(recorded_dttm_min, '%Y%m%d%H%M') AS combo_id
    FROM vit where recorded_dttm_min is not null 
)
PIVOT vital_data
ON vital_category
USING first(vital_value)
GROUP BY combo_id
"""
p_vitals = duckdb.sql(query).df()

#### id-ing all unique timestamps

In [39]:
duckdb.register("expanded_df", wide_cohort_df)
duckdb.register("p_pas", p_pas)
duckdb.register("p_mac", p_mac)

q="""
  WITH
    u_rst as (
        select
            *,
            hospitalization_id || '_' || strftime (recorded_dttm, '%Y%m%d%H%M') AS combo_id
        from
            rst
    ),
    u_adt as (
        select
            *,
            hospitalization_id || '_' || strftime (in_dttm, '%Y%m%d%H%M') AS combo_id
        from
            adt
    ),
    u_expanded_df as (
        select
            *,
            hospitalization_id || '_' || strftime (event_time, '%Y%m%d%H%M') AS combo_id
        from
            expanded_df
    )
select
    *
from
    u_expanded_df a
    left join u_adt d on a.combo_id = d.combo_id
    left join u_rst e on a.combo_id = e.combo_id
    left join p_mac g on a.combo_id = g.combo_id
    left join p_pas h on a.combo_id = h.combo_id
    left join p_vitals i on a.combo_id=i.combo_id 

                    
"""

all_join_df = duckdb.sql(q).df().drop_duplicates()

In [40]:
all_join_df.shape

(1824966, 66)

In [41]:
wide_cohort_df.shape

(1824960, 11)

In [42]:
# all_join_df.drop(columns= ['hospitalization_id_2','hospitalization_id_3','combo_id', 'combo_id_2' ,'combo_id_3','combo_id_4','combo_id_5','recorded_dttm','combo_id_6','in_dttm'], axis = 1,inplace=True)

all_join_df["event_time"] = pd.to_datetime(all_join_df["event_time"])
all_join_df["date"] = all_join_df["event_time"].dt.date

all_join_df = all_join_df.sort_values(["hospitalization_id", "event_time"]).reset_index(
    drop=True
)

# Assign day numbers to each 'hospitalization_id'
all_join_df["day_number"] = (
    all_join_df.groupby("hospitalization_id")["date"].rank(method="dense").astype(int)
)

# Create the combo_key by combining 'hospitalization_id' and 'day_number'
all_join_df["hosp_id_day_key"] = (
    all_join_df["hospitalization_id"].astype(str)
    + "_day_"
    + all_join_df["day_number"].astype(str)
)

In [43]:
##SAT SBT columns check

columns_to_check = [
    "sbt_delivery_pass_fail",
    "sbt_screen_pass_fail",
    "sat_delivery_pass_fail",
    "sat_screen_pass_fail",
    "rass",
    "gcs_total",
]

for col in columns_to_check:
    if col not in all_join_df.columns:
        all_join_df[col] = np.nan
        print(
            f"Column '{col}' is missing for your site. Filling with NaN for now. If this is unintended, please verify your data element."
        )

## meds check

meds_check = [
    "norepinephrine",
    "epinephrine",
    "phenylephrine",
    "angiotensin",
    "vasopressin",
    "dopamine",
    "dobutamine",
    "milrinone",
    "isoproterenol",
    "cisatracurium",
    "vecuronium",
    "rocuronium",
    "fentanyl",
    "propofol",
    "lorazepam",
    "midazolam",
    "hydromorphone",
    "morphine",
]

for col in meds_check:
    if col not in all_join_df.columns:
        all_join_df[col] = np.nan
        print(
            f"mCide: '{col}' is missing. Please check your CLIF Meds table, it's can be missing if your site doesn't use it."
        )

mCide: 'angiotensin' is missing. Please check your CLIF Meds table, it's can be missing if your site doesn't use it.
mCide: 'vecuronium' is missing. Please check your CLIF Meds table, it's can be missing if your site doesn't use it.
mCide: 'rocuronium' is missing. Please check your CLIF Meds table, it's can be missing if your site doesn't use it.


In [44]:
all_join_df.to_csv('../output/intermediate/study_cohort.csv', index=False)

In [ ]:
directory_path = os.path.join("../output/final/", pc.helper["site_name"])
# Create the directory if it doesn't exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' created.")
else:
    print(f"Directory '{directory_path}' already exists.")

Directory '../output/final/RUSH' already exists.


In [45]:
print('cohort creation completed!')

cohort creation completed!
